Exploration of making a map to plot late calls for the tornado and derecho storm for Nashville Metro E911.
Notebook by Chris Mulvey as part of a project for Data Analysis Boot Camp at Nashville Software School.
12 November 2020.

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
derecho_calls = pd.read_csv('../data/911_Phone Calls_Derecho_050320 050420.csv')
derecho_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420.csv',
                               skiprows = 3)
tornado_calls = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
tornado_computer = pd.read_csv('../data/Computer Aided Dispatch Data_Tornado Incidents 030320.csv', skiprows = 3)

In [3]:
derecho_calls.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,5/3/2020 16:30,911 Calls,36.247802,-86.716847,WIRELESS CALLER
1,5/3/2020 16:30,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE
2,5/3/2020 16:30,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N
3,5/3/2020 16:30,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW
4,5/3/2020 16:31,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE


In [4]:
derecho_computer.head()

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1
0,5/3/2020 4:31:07 PM,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD
1,5/3/2020 4:32:18 PM,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD
2,5/3/2020 4:32:23 PM,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD
3,5/3/2020 4:33:04 PM,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN
4,5/3/2020 4:33:52 PM,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD


In [5]:
tornado_calls.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector
1,3/3/2020 0:36,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE
2,3/3/2020 0:36,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector
3,3/3/2020 0:36,911 Calls,NaN,NaN,526 FOUNDRY DR
4,3/3/2020 0:36,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N


In [6]:
tornado_computer.head()

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
1,3/3/2020 12:37:24 AM,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
2,3/3/2020 12:38:17 AM,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL
3,3/3/2020 12:39:40 AM,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD
4,3/3/2020 12:39:42 AM,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD


Converting date time from object to timestamp

In [7]:
tornado_calls['Timestamp'] = pd.to_datetime(tornado_calls['Seizure DateTime'])

In [8]:
tornado_calls.head(1)

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,Timestamp
0,3/3/2020 0:34,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,2020-03-03 00:34:00


In [9]:
tornado_computer['Timestamp'] = pd.to_datetime(tornado_computer['IncidentDate'])

In [10]:
tornado_computer.head(1)

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
0,3/3/2020 12:36:34 AM,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,2020-03-03 00:36:34


In [11]:
late_tornado_calls = tornado_calls[tornado_calls['Timestamp'] >= '2020-03-03 02:00:00']

In [12]:
late_tornado_calls.value_counts('Cell Tower Address')

Cell Tower Address
WIRELESS CALLER                        54
1051G CHICAMAUGA AVE - S                6
4914 OLD HICKORY BLVD - S Sector        4
4884 LICKTON PIKE - SE                  4
1608F ROSA L PARKS BLVD - S             3
                                       ..
5060 TIDWELL HOLLOW RD - SW             1
508 INTERSTATE BLVD S - SW              1
5171 HICKORY HOLLOW PKY - SE            1
5171 HICKORY HOLLOW PKY - SE Sector     1
1 TERMINAL DR                           1
Length: 102, dtype: int64

In [13]:
late_tornado_calls.head()

,Seizure DateTime,CallTypeId,ALI Latitude,ALI Longitude,Cell Tower Address,Timestamp
292,3/3/2020 2:00,911 Calls,36.154633,-86.784500,PRELIM 1033 DEMONBREUN ST - SE Sector,2020-03-03 02:00:00
293,3/3/2020 2:00,911 Calls,36.182045,-86.749270,WIRELESS CALLER,2020-03-03 02:00:00
294,3/3/2020 2:00,911 Calls,36.176723,-86.812613,2101-D HEIMAN ST - SECTOR N,2020-03-03 02:00:00
295,3/3/2020 2:00,911 Calls,NaN,NaN,310 N 9TH ST,2020-03-03 02:00:00
296,3/3/2020 2:01,911 Calls,36.183674,-86.624687,4914 OLD HICKORY BLVD - S Sector,2020-03-03 02:01:00


In [14]:
late_tornado_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 292 to 474
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    183 non-null    object        
 1   CallTypeId          183 non-null    object        
 2   ALI Latitude        170 non-null    float64       
 3   ALI Longitude       170 non-null    float64       
 4   Cell Tower Address  183 non-null    object        
 5   Timestamp           183 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 10.0+ KB


In [15]:
late_tornado_comp = tornado_computer[tornado_computer['Timestamp'] >= '2020-03-03 02:00:00']

In [16]:
late_tornado_comp.value_counts('IncidentTypeDescription1')

IncidentTypeDescription1
OUTSIDE ODOR                                             17
WIRES DOWN WITH NO SMOKE OR ARCING                        9
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD               8
ELECTRICAL HAZARD WITH OR NEAR WATER                      7
CITIZEN ASSIST  NON MEDICAL                               6
                                                         ..
ODOR OF SMOKE OUTSIDE                                     1
WIRES DOWN WITH NO SMOKE OR ARCING POWER DISCONNECTED     1
HOUSE NATURAL GAS ODOR                                    1
HOUSE FIRE                                                1
ABDOMINAL PAIN - NOT ALERT                                1
Length: 62, dtype: int64

In [17]:
late_tornado_comp.head()

,IncidentDate,Location,Latitude1,Longitude1,IncidentTypeDescription1,Timestamp
129,3/3/2020 2:01:33 AM,1421 WOODLAND ST,36.177240,-86.743885,SMOKE DETECTOR ALARM HOUSE,2020-03-03 02:01:33
130,3/3/2020 2:01:52 AM,2658 BARCLAY DR,36.174243,-86.719824,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,2020-03-03 02:01:52
131,3/3/2020 2:02:32 AM,S 9TH ST / MAIN ST,36.175964,-86.757246,OUTSIDE NATURAL GAS ODOR COMMERCIAL LINE TANK ...,2020-03-03 02:02:32
132,3/3/2020 2:03:06 AM,111 EVANDER ST,36.175976,-86.740844,OUTSIDE ODOR,2020-03-03 02:03:06
133,3/3/2020 2:04:00 AM,938 W EASTLAND AV,36.184138,-86.754363,OUTSIDE ODOR,2020-03-03 02:04:00


In [18]:
late_tornado_comp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131 entries, 129 to 259
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   IncidentDate              131 non-null    object        
 1   Location                  131 non-null    object        
 2   Latitude1                 131 non-null    float64       
 3   Longitude1                131 non-null    float64       
 4   IncidentTypeDescription1  131 non-null    object        
 5   Timestamp                 131 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 7.2+ KB


In [19]:
# Removing rows with NaNs in latitude and longitude.
late_tornado_calls = late_tornado_calls.dropna(axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])
late_tornado_comp = late_tornado_comp.dropna(axis = 0, subset = ['Longitude1', 'Latitude1'])

In [31]:
# Latitude and Longitude for Nashville city center from Google.
center = [36.1627, -86.7816]

map_late_tornado = folium.Map(location =  center, zoom_start = 12)

#folium.GeoJson(neighborhoods).add_to(map_neighborhoods)

# Plotting the late tornado calls in red
feature_group = folium.FeatureGroup(name = 'Late Tornado Calls')
for row_index, row_values in late_tornado_calls.iterrows():
    loc1 = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop1 = row_values['Seizure DateTime'], row_values['Cell Tower Address']#, row_values['name']
    icon1 = folium.Icon(color = "red", icon = "phone-square", prefix = 'fa')
    marker1 = folium.Marker(
        location = loc1, 
        popup = pop1, icon = icon1)
    
    marker1.add_to(feature_group)
feature_group.add_to(map_late_tornado)  

# Plotting the derecho dispatches in green    
feature_group1 = folium.FeatureGroup(name = 'Late Tornado Dispatch')
for row_index, row_values in late_tornado_comp.iterrows():
    loc2 = [row_values['Latitude1'], row_values['Longitude1']]
    pop2 = row_values['IncidentDate'], row_values['IncidentTypeDescription1']#, row_values['name']
    icon2 = folium.Icon(color = 'green', icon = 'desktop', prefix = 'fa')
    marker2 = folium.Marker(
        location = loc2, 
        popup = pop2, icon = icon2)
    
    marker2.add_to(feature_group1)
feature_group1.add_to(map_late_tornado)

# Adding layer controls to map
folium.LayerControl().add_to(map_late_tornado)

map_late_tornado.save('../maps/late_tornado.html')

map_late_tornado

In [21]:
derecho_calls['Timestamp'] = pd.to_datetime(derecho_calls['Seizure DateTime'])

In [22]:
late_derecho_calls = derecho_calls[derecho_calls['Timestamp'] >= '2020-05-03 20:00:00']

In [23]:
late_derecho_calls.value_counts('Cell Tower Address')

Cell Tower Address
WIRELESS CALLER                   125
3901 GRANNY WHITE PIKE - SE         8
5801C OLD HARDING RD - SE           6
WIRELESS 911 CALLER                 6
5646 AMALIE DR - SE                 6
                                 ... 
409 LAFAYETTE ST - NE               1
4061 TIMBERVIEW LA - SECTOR SW      1
4061 TIMBERVIEW LA - SECTOR SE      1
405 WOODSMAN CT                     1
10 C LUCILLE ST - N SECTOR          1
Length: 264, dtype: int64

In [24]:
late_derecho_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 1042 to 1548
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    507 non-null    object        
 1   CallTypeId          507 non-null    object        
 2   ALI Latitude        494 non-null    float64       
 3   ALI Longitude       494 non-null    float64       
 4   Cell Tower Address  507 non-null    object        
 5   Timestamp           507 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 27.7+ KB


In [25]:
derecho_computer['Timestamp'] = pd.to_datetime(derecho_computer['IncidentDate'])

In [26]:
late_derecho_comp = derecho_computer[derecho_computer['Timestamp'] >= '2020-05-03 20:00:00']

In [27]:
late_derecho_comp.value_counts('IncidentTypeDescription1')

IncidentTypeDescription1
ELECTRICAL HAZARD                                         96
GENERAL ALARM COMMERCIAL   INDUSTRIAL BUILD                7
SMOKE DETECTOR ALARM HOUSE                                 5
CARBON MONOXIDE ALARM HOUSE                                5
UNKNOWN STATUS:                                            4
                                                          ..
INEFFECTIVE BREATHING COPD                                 1
INEFFECTIVE BREATHING                                      1
HIGH RISE NATURAL GAS ODOR                                 1
HIGH MECHANISM: AUTO VS BICYCLE OR AUTO VS MOTOROCYCLE     1
3RD TRIMESTER HEMORRHAGE                                   1
Length: 76, dtype: int64

In [28]:
late_derecho_calls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507 entries, 1042 to 1548
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Seizure DateTime    507 non-null    object        
 1   CallTypeId          507 non-null    object        
 2   ALI Latitude        494 non-null    float64       
 3   ALI Longitude       494 non-null    float64       
 4   Cell Tower Address  507 non-null    object        
 5   Timestamp           507 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 27.7+ KB


In [29]:
late_derecho_calls = late_derecho_calls.dropna(axis = 0, subset = ['ALI Longitude', 'ALI Latitude'])
late_derecho_comp = late_derecho_comp.dropna(axis = 0, subset = ['Longitude1', 'Latitude1'])

In [32]:
# Latitude and Longitude for Nashville city center from Google.
center = [36.1627, -86.7816]

map_late_derecho = folium.Map(location =  center, zoom_start = 12)

#folium.GeoJson(neighborhoods).add_to(map_neighborhoods)

# Plotting the late tornado calls in red
feature_group = folium.FeatureGroup(name = 'Late Derecho Calls')
for row_index, row_values in late_derecho_calls.iterrows():
    loc1 = [row_values['ALI Latitude'], row_values['ALI Longitude']]
    pop1 = row_values['Seizure DateTime'], row_values['Cell Tower Address']#, row_values['name']
    icon1 = folium.Icon(color = "red", icon = "phone-square", prefix = 'fa')
    marker1 = folium.Marker(
        location = loc1, 
        popup = pop1, icon = icon1)
    
    marker1.add_to(feature_group)
feature_group.add_to(map_late_derecho)  

# Plotting the derecho dispatches in green    
feature_group1 = folium.FeatureGroup(name = 'Late Derecho Dispatch')
for row_index, row_values in late_derecho_comp.iterrows():
    loc2 = [row_values['Latitude1'], row_values['Longitude1']]
    pop2 = row_values['IncidentDate'], row_values['IncidentTypeDescription1']#, row_values['name']
    icon2 = folium.Icon(color = 'green', icon = 'desktop', prefix = 'fa')
    marker2 = folium.Marker(
        location = loc2, 
        popup = pop2, icon = icon2)
    
    marker2.add_to(feature_group1)
feature_group1.add_to(map_late_derecho)
    
# Adding layer controls to map
folium.LayerControl().add_to(map_late_derecho)

map_late_derecho.save('../maps/late_derecho.html')

map_late_derecho